In [159]:
import pandas as pd
# from ydata_profiling import ProfileReport

In [160]:
df = pd.read_csv('csv/uspevaemost.csv', encoding='cp1251')


In [161]:
df[df['Студент'] == '577856417A727062382F68576E466E513459706833673D3D'].sort_values(by=['Учебный год', 'Семестр'])

,Студент,Группа,Дисциплина,Семестр,Учебный год,Оценка,Специальность,Форма обучения,Квалификация,Статус
137898,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,История,1,2014/2015,хорошо,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
137899,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Иностранный язык (русский),1,2014/2015,зачтено,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
138016,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Математика 1.1,1,2014/2015,удовлетворительно,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
138017,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Информатика 1.1,1,2014/2015,зачтено,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
138018,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Химия 1.2,1,2014/2015,хорошо,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
...,...,...,...,...,...,...,...,...,...,...
180657,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Учебно-исследовательская работа студентов,6,2017/2018,неудовлетворительно,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
181363,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Экономика 2.5,6,2017/2018,незачет,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
181364,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Экономика 2.5,6,2017/2018,неудовлетворительно,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный
181754,577856417A727062382F68576E466E513459706833673D3D,6643334D2F79793236556237426F39725945366F61413D3D,Математические основы обработки сигналов,6,2017/2018,незачет,12.03.01 Приборостроение,Очная,Бакалавр,Отчисленный


In [162]:
df

,Студент,Группа,Дисциплина,Семестр,Учебный год,Оценка,Специальность,Форма обучения,Квалификация,Статус
0,4B6E636B427939346D53424141516C386C54354277413D3D,585342585836624C3664486C6F456175377A626E6F773D3D,Микропроцессорная техника и встраиваемые системы,7,2018/2019,хорошо,15.03.04 Автоматизация технологических процесс...,Заочная,Бакалавр,Отчисленный
1,6156577743516E3279686B794A626430754A4B7042673D3D,585342585836624C3664486C6F456175377A626E6F773D3D,Микропроцессорная техника и встраиваемые системы,7,2018/2019,хорошо,15.03.04 Автоматизация технологических процесс...,Заочная,Бакалавр,Отчисленный
2,4764566C52634E5763356E304E713765324469705A673D3D,585342585836624C3664486C6F456175377A626E6F773D3D,Микропроцессорная техника и встраиваемые системы,7,2018/2019,удовлетворительно,15.03.04 Автоматизация технологических процесс...,Заочная,Бакалавр,Отчисленный
3,725572452B54307A42546F356C713957795064657A413D3D,585342585836624C3664486C6F456175377A626E6F773D3D,Микропроцессорная техника и встраиваемые системы,7,2018/2019,отлично,15.03.04 Автоматизация технологических процесс...,Заочная,Бакалавр,Отчисленный
4,5A77455046706A6133496E47654D6334646369702F673D3D,585342585836624C3664486C6F456175377A626E6F773D3D,Микропроцессорная техника и встраиваемые системы,7,2018/2019,хорошо,15.03.04 Автоматизация технологических процесс...,Заочная,Бакалавр,Отчисленный
...,...,...,...,...,...,...,...,...,...,...
234835,754139666F566F64514C4D79525168476875767177673D3D,694672466B38596E6767575457613968565833394E673D3D,Учебно-исследовательская работа студентов,10,2019/2020,зачтено,21.05.04 Горное дело,Очная,Специалист,Учащийся
234836,706F47734F74715753413259726572547636684544513D3D,6F4F686974634C6541314B757546485A512B594562413D3D,Механика жидкости и газа,7,2018/2019,зачтено,21.03.01 Нефтегазовое дело,Заочная,Бакалавр,Отчисленный
234837,4B4E3450494461702F393850546966716137374B35413D3D,74753964553444596A4F356B5A547851734A697952513D3D,Информатика 1.2,1,2015/2016,зачтено,09.03.01 Информатика и вычислительная техника,Заочная,Бакалавр,Отчисленный
234838,706F47734F74715753413259726572547636684544513D3D,6F4F686974634C6541314B757546485A512B594562413D3D,Начертательная геометрия и инженерная графика 1.4,2,2015/2016,удовлетворительно,21.03.01 Нефтегазовое дело,Заочная,Бакалавр,Отчисленный


In [163]:
df = df[df['Статус'] == 'Отчисленный']
df = df[df['Квалификация'] == 'Бакалавр']
df = df[df['Форма обучения'] == 'Очная']

In [164]:
df['sem_start'] = df['Учебный год'].str[:4]
df['sem_end'] = df['Учебный год'].str[5:]

df_years = df.groupby(['Студент']).agg({'sem_start': 'min', 'sem_end': 'max'})

df_kol_let = (df_years['sem_end']).astype('int16') - (df_years['sem_start']).astype('int16')
print(df_kol_let.shape)
(df_kol_let <= 4).sum()

df_kol_let = df_kol_let[df_kol_let > 4]


# df_kol_let = pd.read_csv('TempData.csv')



(1536,)


In [165]:
df_kol_let.to_frame()

df_kol_let.to_csv('TempData.csv', index=True)
df_kol_let = pd.read_csv('TempData.csv')

df_kol_let.drop(df_kol_let.columns[1], axis=1, inplace=True)

df = df.merge(df_kol_let, indicator=True, how='outer').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge']).drop_duplicates()




In [166]:
df.to_csv('TempData1.csv', index=False)

df.drop_duplicates(inplace=True)

df = df[~(df['Оценка'] == 'неудовлетворительно') | 
        ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

In [167]:
df.drop('Статус', axis=1, inplace=True)
df.drop('Группа', axis=1, inplace=True)
df.drop('Специальность', axis=1, inplace=True)
df.drop('Квалификация', axis=1, inplace=True)

In [168]:
df_max = df.groupby(['Студент', 'Учебный год', 'Семестр', 'Дисциплина']).agg({'Оценка': 'max'})
df_min = df.groupby(['Студент', 'Учебный год',  'Семестр', 'Дисциплина']).agg({'Оценка': 'min'})

# df.sort_values(by=['Студент',"Учебный год", 'Семестр'], inplace=True)

In [169]:
df_max

Оценка
Студент                                          Учебный год Семестр Дисциплина                                                
2B41566C72437872494D356F7673772B4731724C4D413D3D 2015/2016   1       Введение в инженерную деятельность                       2
                                                                     Иностранный язык (английский)                            2
                                                                     Информатика 1.1                                          2
                                                                     История                                                  4
                                                                     Математика 1.1                                           3
...                                                                                                                         ...
7A7A38456B32345073425950566354506C2F4E3949773D3D 2015/2016   1       Математика 1.1                                           1
                                                                     Начертательная геометрия и инженерная графика 1.3        1
                                                                     Основы ядерных технологий                                2
                                                                     Химия 1.2                                                4
                                                                     Элективные дисциплины по физической культуре и ...       2

[78866 rows x 1 columns]

In [170]:
df_min = df_min.groupby(['Студент', 'Учебный год', "Семестр"], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_max = df_max.groupby(['Студент', 'Учебный год', "Семестр"], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)

In [171]:
df = df_max.merge(df_min, on=['Студент', 'Учебный год', 'Семестр']).rename(columns={'1_x': '1_max', '2_x': '2_max', '3_x': '3_max', '4_x': '4_max', '5_x': '5_max', 
                                                                                    '1_y': '1_min', '2_y': '2_min', '3_y': '3_min', '4_y': '4_min', '5_y': '5_min'})

df.to_csv('test1.csv', index=True)
df = pd.read_csv('test1.csv')
df

,Студент,Учебный год,Семестр,1_max,2_max,3_max,4_max,5_max,1_min,2_min,3_min,4_min,5_min
0,2B41566C72437872494D356F7673772B4731724C4D413D3D,2015/2016,1,0,4,2,2,0,0,4,2,2,0
1,2B41566C72437872494D356F7673772B4731724C4D413D3D,2015/2016,2,0,7,1,2,0,0,7,1,2,0
2,2B41566C72437872494D356F7673772B4731724C4D413D3D,2016/2017,3,0,5,2,2,0,0,5,2,2,0
3,2B41566C72437872494D356F7673772B4731724C4D413D3D,2016/2017,4,0,5,1,4,0,0,6,1,3,0
4,2B41566C72437872494D356F7673772B4731724C4D413D3D,2017/2018,5,0,5,1,3,0,0,5,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8955,7A74794B6C504F644431394D5A652F517757433847413D3D,2017/2018,5,0,5,4,0,0,0,5,4,0,0
8956,7A74794B6C504F644431394D5A652F517757433847413D3D,2017/2018,6,0,5,3,1,0,0,5,3,1,0
8957,7A74794B6C504F644431394D5A652F517757433847413D3D,2018/2019,7,0,5,2,2,0,0,5,4,0,0
8958,7A74794B6C504F644431394D5A652F517757433847413D3D,2018/2019,8,0,3,0,3,0,0,3,0,3,0


In [172]:
df['количество'] = df.groupby(['Студент', 'Учебный год'])['Семестр'].transform('count')

In [111]:
df = df[df['Студент'] == '577856417A727062382F68576E466E513459706833673D3D']

In [173]:
df = df[df['количество'] > 2]

In [174]:
df.head(20)

,Студент,Учебный год,Семестр,1_max,2_max,3_max,4_max,5_max,1_min,2_min,3_min,4_min,5_min,количество
2218,45635A5A4E355768674A794B68664D52596961472F673D3D,2016/2017,3,3,5,1,0,0,3,5,1,0,0,3
2219,45635A5A4E355768674A794B68664D52596961472F673D3D,2016/2017,4,1,0,0,0,0,1,0,0,0,0,3
2220,45635A5A4E355768674A794B68664D52596961472F673D3D,2016/2017,6,1,0,0,0,0,1,0,0,0,0,3
2784,4A5A34517248624747432F4F435765697037444469673D3D,2017/2018,5,0,9,0,2,3,0,10,0,2,2,3
2785,4A5A34517248624747432F4F435765697037444469673D3D,2017/2018,6,0,8,0,1,3,0,8,0,2,2,3
2786,4A5A34517248624747432F4F435765697037444469673D3D,2017/2018,7,0,1,0,0,0,0,1,0,0,0,3
3100,4B70534F4C7153537767426A4D36314F55376E5477513D3D,2014/2015,2,0,2,0,0,0,0,2,0,0,0,3
3101,4B70534F4C7153537767426A4D36314F55376E5477513D3D,2014/2015,3,0,3,6,0,0,0,3,6,0,0,3
3102,4B70534F4C7153537767426A4D36314F55376E5477513D3D,2014/2015,4,0,6,2,1,0,0,6,2,1,0,3
4704,577856417A727062382F68576E466E513459706833673D3D,2015/2016,1,1,3,1,2,0,1,3,1,2,0,4


In [ ]:
df.to_csv('test.csv', index=True)